In [10]:
import re
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import jieba
import jieba.analyse
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.model_selection import train_test_split

# url = 'https://movies.yahoo.com.tw/movie_thisweek.html'

# response = requests.get(url=url)

# soup = BeautifulSoup(response.text, 'lxml')

# info_items = soup.find_all('div', 'release_info')

# print("HW3 test")

# with open('本週新片.csv', 'w', encoding='big5', newline='') as csv_file:
    
#     csv_writer = csv.writer(csv_file)
#     csv_writer.writerow(['電影片名', '電影英文片名', '上映時間', '網友期待度'])

#     for item in info_items:

#         name = item.find('div', 'release_movie_name').a.text.strip()
#         english_name = item.find('div', 'en').a.text.strip()
#         release_time = item.find('div', 'release_movie_time').text.split('：')[-1].strip()
#         level = item.find('div', 'leveltext').span.text.strip()
        
#         csv_writer.writerow([name, english_name, release_time, level])
#         print('{}({}) 上映日：{} 期待度：{}'.format(name, english_name, release_time, level))

In [9]:
def cutval(df):
    topcut = []
    for i in df['簡介']:
        top = jieba.analyse.extract_tags(i,topK=2)
        topcut.append(top)

    typecut = []
    for i in df['類型']:
        ch =re.compile("[\u4e00-\u9fa5]")
        seg_word =  "".join(ch.findall(i))
        top = jieba.lcut(seg_word)
        typecut.append(top)

    namecut = []
    for i in df['中文名稱']:
        ch =re.compile("[\u4e00-\u9fa5]")
        name =  "".join(ch.findall(i))
        namecut.append(name)

    data = { 'type':typecut ,'name':namecut,'article':topcut}
    df1 = pd.DataFrame(data)

    df1['type'] = df1['type'].apply(lambda x :  str(x)[1:-2].replace("'","").replace(",",""))
    df1['article'] = df1['article'].apply(lambda x :  str(x)[1:-2].replace("'","").replace(",",""))
    df1['key'] = df1['name'].astype(str)+' '+df1['type'].astype(str)+' '+df1['article'].astype(str)
    return df1

In [5]:
def count(df):
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(df['key'])
    tfidf = TfidfTransformer() 
    tf=tfidf.fit_transform(X)
    word = vectorizer.get_feature_names() #詞袋 
    
    return tf

In [6]:
def knn(X,Y):
    X_train, X_test, y_train, y_test = train_test_split(X, Y['type'].str[0:2], test_size=0.07659)
    clf=KNeighborsClassifier(n_neighbors=51)
    clf.fit(X_train,y_train)

    y_pred = clf.predict(X_test)    # 預測模型 
    y_test=y_test.values

    print("精準度：",metrics.accuracy_score(y_test, y_pred))

In [8]:
df = pd.read_csv('movie.csv')
df = df.drop(labels=['Unnamed: 0'],axis='columns')
df1 = cutval(df) 
tf = count(df1)
knn(tf,df1)

Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\User\AppData\Local\Temp\jieba.cache
Loading model cost 0.537 seconds.
Prefix dict has been built successfully.


精準度： 0.758
